In [1]:
from fastapi import FastAPI, UploadFile, File
from fastapi.param_functions import Depends
from pydantic import BaseModel, Field
from uuid import UUID, uuid4
from typing import List, Union, Optional, Dict, Any

from datetime import datetime
from app.firebase_db import load_database, get_user_solved, get_dataframe

import requests
from app.model import load_data, tfidf_recommender

app = FastAPI()
db = load_database()
df = get_dataframe(db)


In [2]:
df

,testMode,userUID,isCorrect,problemCode,selected,solvedAt,timetaken
0,True,LH8ERBe1DpXavxtNNxKE0KIPyio2,True,advanced4701,1,1672884339704,2606
1,True,LH8ERBe1DpXavxtNNxKE0KIPyio2,True,advanced4702,2,1672884339704,1511
2,True,LH8ERBe1DpXavxtNNxKE0KIPyio2,True,advanced4703,4,1672884339704,1490
3,True,LH8ERBe1DpXavxtNNxKE0KIPyio2,True,advanced4704,1,1672884339704,1414
4,True,LH8ERBe1DpXavxtNNxKE0KIPyio2,False,advanced4705,5,1672884339704,1161
...,...,...,...,...,...,...,...
6467,False,e9pNYfHjZjdZnN2Vsq9rMuGFsB02,True,basic5048,1,1675129698249,3379
6468,False,nNInpJaEq7bhWd0s9kvL8qNvnFn1,False,basic5013,3,1675129702319,49718
6469,False,e9pNYfHjZjdZnN2Vsq9rMuGFsB02,True,basic5204,2,1675129837483,135182
6470,False,nNInpJaEq7bhWd0s9kvL8qNvnFn1,False,basic4932,3,1675129904303,52011


In [2]:
df

,problemCode,userUID,timetaken,isCorrect,selected,solvedAt,testMode
0,advanced4701,LH8ERBe1DpXavxtNNxKE0KIPyio2,2606,True,1,1672884339704,True
1,advanced4702,LH8ERBe1DpXavxtNNxKE0KIPyio2,1511,True,2,1672884339704,True
2,advanced4703,LH8ERBe1DpXavxtNNxKE0KIPyio2,1490,True,4,1672884339704,True
3,advanced4704,LH8ERBe1DpXavxtNNxKE0KIPyio2,1414,True,1,1672884339704,True
4,advanced4705,LH8ERBe1DpXavxtNNxKE0KIPyio2,1161,False,5,1672884339704,True
...,...,...,...,...,...,...,...
5050,basic5426,xvTAj5nNiGcIGAwBki19UMC6JQw1,470419,True,2,1675060426021,False
5051,basic6125,xvTAj5nNiGcIGAwBki19UMC6JQw1,4327,False,4,1675060433728,False
5052,basic5147,xvTAj5nNiGcIGAwBki19UMC6JQw1,2099,False,4,1675060497575,False
5053,basic4710,xvTAj5nNiGcIGAwBki19UMC6JQw1,2603,False,3,1675060501114,False


In [4]:
# df.to_csv('han_df.csv', index=None)

In [2]:
df

NameError: name 'df' is not defined

In [3]:
df['isCorrect'] = df['isCorrect'].apply(int)
df['testMode'] = df['testMode'].apply(int)
df

,solvedAt,problemCode,isCorrect,userUID,selected,timetaken,testMode
0,1672884339704,advanced4701,1,LH8ERBe1DpXavxtNNxKE0KIPyio2,1,2606,1
1,1672884339704,advanced4702,1,LH8ERBe1DpXavxtNNxKE0KIPyio2,2,1511,1
2,1672884339704,advanced4703,1,LH8ERBe1DpXavxtNNxKE0KIPyio2,4,1490,1
3,1672884339704,advanced4704,1,LH8ERBe1DpXavxtNNxKE0KIPyio2,1,1414,1
4,1672884339704,advanced4705,0,LH8ERBe1DpXavxtNNxKE0KIPyio2,5,1161,1
...,...,...,...,...,...,...,...
5038,1675057411252,basic5436,0,e9pNYfHjZjdZnN2Vsq9rMuGFsB02,1,8899,1
5039,1675057419719,basic4849,0,e9pNYfHjZjdZnN2Vsq9rMuGFsB02,4,6826,1
5040,1675057470535,basic5107,1,e9pNYfHjZjdZnN2Vsq9rMuGFsB02,2,4743,1
5041,1675057491887,basic5240,0,e9pNYfHjZjdZnN2Vsq9rMuGFsB02,4,16396,1


In [3]:
df_sel = df.loc[:, ['userUID', 'problemCode', 'isCorrect', 'solvedAt']]
df_sel['isCorrect'] = df_sel['isCorrect'].apply(int)

df_sel = df_sel.rename(columns={'userUID':'user_id',
                'problemCode':'item_id',
                'isCorrect':'rating',
                'solvedAt':'timestamp'
                })

df_sel
# 0 - 1 switching
df_sel['rating'] = df_sel['rating'].apply(lambda x: 0 if x==1 else 1)
df_sel

,user_id,item_id,rating,timestamp
0,LH8ERBe1DpXavxtNNxKE0KIPyio2,advanced4701,0,1672884339704
1,LH8ERBe1DpXavxtNNxKE0KIPyio2,advanced4702,0,1672884339704
2,LH8ERBe1DpXavxtNNxKE0KIPyio2,advanced4703,0,1672884339704
3,LH8ERBe1DpXavxtNNxKE0KIPyio2,advanced4704,0,1672884339704
4,LH8ERBe1DpXavxtNNxKE0KIPyio2,advanced4705,1,1672884339704
...,...,...,...,...
6467,e9pNYfHjZjdZnN2Vsq9rMuGFsB02,basic5048,0,1675129698249
6468,nNInpJaEq7bhWd0s9kvL8qNvnFn1,basic5013,1,1675129702319
6469,e9pNYfHjZjdZnN2Vsq9rMuGFsB02,basic5204,0,1675129837483
6470,nNInpJaEq7bhWd0s9kvL8qNvnFn1,basic4932,1,1675129904303


In [10]:
df_sel.to_csv('han_df_sel.csv', sep='\t', index=None)

In [13]:
df.to_csv('han_df.csv', index=None)

In [5]:
user_list = df.userUID.unique()
len(user_list)

76

## 데이터 관련 TODOs

- 모델링
    - DKT 모델 시도 : Recbole 이용

- 데이터 분석
    - 유저당 푼 문제 수 시각화
    - dummy 유저 필터링

- 데이터 추가
    - 문제 정보를 더 주어야 모델의 성능을 높일 수 있음.
    - 어떤 feature를 주어야 할까?
    - 문제 대분류, 소분류, 유형 나누기
    
    
    

In [8]:
df.iloc[100]

isCorrect                                 1
selected                                  4
solvedAt                      1673412302113
userUID        LH8ERBe1DpXavxtNNxKE0KIPyio2
testMode                                  1
timetaken                              1556
problemCode                       basic4703
level                                 basic
Name: 100, dtype: object

In [10]:
df['level'] = df['problemCode'].apply(lambda x: 'advanced' if x.startswith('a') else 'basic')
df['testnum'] = df['problemCode'].str[-4:-2] 
df['probnum'] = df['problemCode'].str[-2:]
df

,isCorrect,selected,solvedAt,userUID,testMode,timetaken,problemCode,level,testnum,probnum
0,1,1,1672884339704,LH8ERBe1DpXavxtNNxKE0KIPyio2,1,2606,advanced4701,advanced,47,01
1,1,2,1672884339704,LH8ERBe1DpXavxtNNxKE0KIPyio2,1,1511,advanced4702,advanced,47,02
2,1,4,1672884339704,LH8ERBe1DpXavxtNNxKE0KIPyio2,1,1490,advanced4703,advanced,47,03
3,1,1,1672884339704,LH8ERBe1DpXavxtNNxKE0KIPyio2,1,1414,advanced4704,advanced,47,04
4,0,5,1672884339704,LH8ERBe1DpXavxtNNxKE0KIPyio2,1,1161,advanced4705,advanced,47,05
...,...,...,...,...,...,...,...,...,...,...
3326,0,4,1674794085187,XbNbgaq12xWNCF3lLBxYeGvDz403,1,17227,advanced4846,advanced,48,46
3327,0,4,1674794085187,XbNbgaq12xWNCF3lLBxYeGvDz403,1,25210,advanced4847,advanced,48,47
3328,1,2,1674794085187,XbNbgaq12xWNCF3lLBxYeGvDz403,1,9237,advanced4848,advanced,48,48
3329,0,2,1674794085187,XbNbgaq12xWNCF3lLBxYeGvDz403,1,30361,advanced4849,advanced,48,49


In [4]:
# 한문제만 푼 유저는 제거
problems_user_solved = df_sel['user_id'].value_counts()
user_over1 = problems_user_solved[problems_user_solved > 1].index
len(user_over1)

df_sel_over1 = df_sel.set_index('user_id').loc[user_over1, :].reset_index()
df_sel_over1

,user_id,item_id,rating,timestamp
0,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5801,0,1674697357662
1,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5802,0,1674697357662
2,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5803,0,1674697357662
3,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5804,0,1674697357662
4,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5805,1,1674697357662
...,...,...,...,...
6458,FNy1tihai6ZvZTAgFTk8NC8STNf1,basic6102,1,1673484516547
6459,KZue2oDp4AP0tJ2hFTM5oMrB2BC3,advanced4701,1,1673532811159
6460,KZue2oDp4AP0tJ2hFTM5oMrB2BC3,advanced4702,1,1673532811159
6461,p0bW29JzcQPMQfgpvUlSiaLLamn2,basic6101,1,1673505128515


In [5]:
df_dd = df_sel_over1.drop_duplicates(subset=['user_id', 'item_id'],
                                keep='last')

df_dd

,user_id,item_id,rating,timestamp
1,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5802,0,1674697357662
2,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5803,0,1674697357662
5,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5806,0,1674697357662
21,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5822,0,1674697357662
22,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5823,0,1674697357662
...,...,...,...,...
6458,FNy1tihai6ZvZTAgFTk8NC8STNf1,basic6102,1,1673484516547
6459,KZue2oDp4AP0tJ2hFTM5oMrB2BC3,advanced4701,1,1673532811159
6460,KZue2oDp4AP0tJ2hFTM5oMrB2BC3,advanced4702,1,1673532811159
6461,p0bW29JzcQPMQfgpvUlSiaLLamn2,basic6101,1,1673505128515


2번 이상 푼 문제는 1005개. 1400개 중 72% 정도이다.

In [6]:
df_eval = df_dd.drop_duplicates(subset=['user_id'], keep='last')
df_eval

,user_id,item_id,rating,timestamp
1045,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5045,1,1675111914602
1659,GOlauafiGceNr8ScPFYQCgW6TAS2,advanced5450,0,1675098353507
2212,e9pNYfHjZjdZnN2Vsq9rMuGFsB02,basic5204,0,1675129837483
2737,LH8ERBe1DpXavxtNNxKE0KIPyio2,advanced4704,1,1675130258788
3098,AmxY7h9SXJPwgMuO9ZfRJMAJcFu2,advanced5924,1,1674924975050
...,...,...,...,...
6454,tbvwLfZbYMYgswPqvwx5UjAUzom2,advanced6127,0,1673405285720
6456,ZNT3IlALSLeEiZaxxhzotvMdOHi2,basic4701,1,1674124089272
6458,FNy1tihai6ZvZTAgFTk8NC8STNf1,basic6102,1,1673484516547
6460,KZue2oDp4AP0tJ2hFTM5oMrB2BC3,advanced4702,1,1673532811159


In [7]:
df_train = df_dd.drop(index = df_eval.index)
df_train

,user_id,item_id,rating,timestamp
1,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5802,0,1674697357662
2,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5803,0,1674697357662
5,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5806,0,1674697357662
21,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5822,0,1674697357662
22,yuAW8RS6gFSLAmpQ2FQLxcXxChs1,advanced5823,0,1674697357662
...,...,...,...,...
6451,wHggVALBIXaywHnbSqgpvqRrnG62,basic5506,1,1673431160513
6453,tbvwLfZbYMYgswPqvwx5UjAUzom2,advanced5636,0,1673405275571
6457,FNy1tihai6ZvZTAgFTk8NC8STNf1,basic6101,1,1673484516547
6459,KZue2oDp4AP0tJ2hFTM5oMrB2BC3,advanced4701,1,1673532811159


In [8]:
print('전체 데이터 수:', len(df_dd))
print('train 데이터 수:', len(df_train))
print('eval 데이터 수:', len(df_eval))

전체 데이터 수: 4810
train 데이터 수: 4721
eval 데이터 수: 89


In [9]:
from logging import getLogger
import os
import json
import pandas as pd
import time, datetime

from recbole.model.general_recommender.lightgcn import LightGCN
from recbole.model.general_recommender.slimelastic import SLIMElastic


from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_logger, get_trainer, init_seed, set_color

from recbole.config import Config
from recbole.data import create_dataset

from sklearn.metrics import accuracy_score, roc_auc_score

import torch

In [12]:
import recbole

recbole.__version__

'1.0.1'

In [10]:
userid = sorted(list(set(df_dd['user_id'])))
itemid = sorted(list(set(df_dd['item_id'])))
n_user, n_item = len(userid), len(itemid)

userid_2_index = {v:i        for i,v in enumerate(userid)}
itemid_2_index = {v:i+n_user for i,v in enumerate(itemid)}
id_2_index = dict(userid_2_index, **itemid_2_index)


In [11]:
yamldata = """
USER_ID_FIELD: user_id
ITEM_ID_FIELD: item_id
RATING_FIELD: rating
TIME_FIELD: timestamp

load_col:
    inter: [user_id, item_id, rating, timestamp]

user_inter_num_interval: "[0,inf)"
item_inter_num_interval: "[0,inf)"
val_interval:
    rating: "[0,1]"
    timestamp: "[1672892980000, inf)"
"""

In [12]:
outpath = f"dataset/train_data"
outfile = f"dataset/train_data/train_data.inter"
yamlfile = f"train_data.yaml"

os.makedirs(outpath, exist_ok=True)

print("Processing Start")
inter_table = []
for user, item, acode, tstamp in zip(df_train.user_id, df_train.item_id, df_train.rating, df_train.timestamp):
    uid, iid = id_2_index[user], id_2_index[item]
    # tval = int(time.mktime(datetime.datetime.strptime(tstamp, "%Y-%m-%d %H:%M:%S").timetuple()))
    inter_table.append( [uid, iid, max(acode,0), tstamp] )

print("Processing Complete")

print("Dump Start")
# 데이터 설정 파일 저장
with open(yamlfile, "w") as f:
    f.write(yamldata) 

# 데이터 파일 저장
with open(outfile, "w") as f:
    # write header
    f.write("user_id:token\titem_id:token\trating:float\ttimestamp:float\n")
    for row in inter_table:
        f.write("\t".join([str(x) for x in row])+"\n")

print("Dump Complete")

Processing Start
Processing Complete
Dump Start
Dump Complete


In [13]:
outpath = f"dataset/test_data"
outfile = f"dataset/test_data/test_data.inter"
yamlfile = f"test_data.yaml"

os.makedirs(outpath, exist_ok=True)

print("Processing Start")
inter_table = []
for user, item, acode, tstamp in zip(df_eval.user_id, df_eval.item_id, df_eval.rating, df_eval.timestamp):
    uid, iid = id_2_index[user], id_2_index[item]
    # tval = int(time.mktime(datetime.datetime.strptime(tstamp, "%Y-%m-%d %H:%M:%S").timetuple()))
    inter_table.append( [uid, iid, max(acode,0), tstamp] )

print("Processing Complete")

print("Dump Start")
# 데이터 설정 파일 저장
with open(yamlfile, "w") as f:
    f.write(yamldata) 

# 데이터 파일 저장
with open(outfile, "w") as f:
    # write header
    f.write("user_id:token\titem_id:token\trating:float\ttimestamp:float\n")
    for row in inter_table:
        f.write("\t".join([str(x) for x in row])+"\n")

print("Dump Complete")

Processing Start
Processing Complete
Dump Start
Dump Complete


In [14]:
logger = getLogger()

### 설정 인스턴스 생성

In [15]:
config = Config(model='LightGCN', dataset='train_data', config_file_list=[f'train_data.yaml'])
config['epochs'] = 100
config['valid_metric'] = 'Recall@10'
config['show_progress'] = False
config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
init_seed(config['seed'], config['reproducibility'])

# logger init
init_logger(config)

logger.info(config)

31 Jan 02:02    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/train_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 100
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = {'uniform': 1}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = Recall@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME

### 데이터 로드

In [16]:
# dataset filtering
dataset = create_dataset(config)
logger.info(dataset)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)


31 Jan 02:02    INFO  train_data
The number of users: 89
Average actions of users: 53.64772727272727
The number of items: 1349
Average actions of items: 3.5022255192878338
The number of inters: 4721
The sparsity of the dataset: 96.0678321853058%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp']
31 Jan 02:02    INFO  [Training]: train_batch_size = [2048] negative sampling: [{'uniform': 1}]
31 Jan 02:02    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}]


### 모델 인스턴스 생성


In [17]:
# model loading and init
init_seed(config['seed'], config['reproducibility'])
model = LightGCN(config, train_data.dataset).to(config['device'])
# model = SLIMElastic(config, train_data.dataset).to(config['device'])
logger.info(model)

/opt/conda/envs/fastapi/lib/python3.8/site-packages/recbole/model/general_recommender/lightgcn.py:103: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  i = torch.LongTensor([row, col])
31 Jan 02:02    INFO  LightGCN(
  (user_embedding): Embedding(89, 64)
  (item_embedding): Embedding(1349, 64)
  (mf_loss): BPRLoss()
  (reg_loss): EmbLoss()
)
Trainable parameters: 92032


### 모델 학습

In [18]:
## LightGCN
### 1-0 역전 - 틀릴 문제 추천하기 위해
# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
# breakpoint()
best_valid_score, best_valid_result = trainer.fit(
    train_data, valid_data, saved=True, show_progress=config['show_progress']
)


31 Jan 02:03    INFO  epoch 0 training [time: 0.04s, train loss: 1.3766]
31 Jan 02:03    INFO  epoch 0 evaluating [time: 0.70s, valid_score: 0.000200]
31 Jan 02:03    INFO  valid result: 
recall@10 : 0.0002    mrr@10 : 0.0046    ndcg@10 : 0.0015    hit@10 : 0.0137    precision@10 : 0.0014
31 Jan 02:03    INFO  Saving current: saved/LightGCN-Jan-31-2023_02-03-12.pth
31 Jan 02:03    INFO  epoch 1 training [time: 0.03s, train loss: 1.3741]
31 Jan 02:03    INFO  epoch 1 evaluating [time: 0.02s, valid_score: 0.003300]
31 Jan 02:03    INFO  valid result: 
recall@10 : 0.0033    mrr@10 : 0.0139    ndcg@10 : 0.0068    hit@10 : 0.0685    precision@10 : 0.0082
31 Jan 02:03    INFO  Saving current: saved/LightGCN-Jan-31-2023_02-03-12.pth
31 Jan 02:03    INFO  epoch 2 training [time: 0.02s, train loss: 1.3719]
31 Jan 02:03    INFO  epoch 2 evaluating [time: 0.02s, valid_score: 0.004300]
31 Jan 02:03    INFO  valid result: 
recall@10 : 0.0043    mrr@10 : 0.0247    ndcg@10 : 0.0095    hit@10 : 0.0822

In [19]:
# model evaluation
test_result = trainer.evaluate(test_data, load_best_model="True", show_progress=config['show_progress'])

logger.info(set_color('best valid ', 'yellow') + f': {best_valid_result}')
logger.info(set_color('test result', 'yellow') + f': {test_result}')

result = {
    'best_valid_score': best_valid_score,
    'valid_score_bigger': config['valid_metric_bigger'],
    'best_valid_result': best_valid_result,
    'test_result': test_result
}

print(json.dumps(result, indent=4))

31 Jan 02:03    INFO  Loading model structure and parameters from saved/LightGCN-Jan-31-2023_02-03-12.pth
31 Jan 02:03    INFO  best valid : OrderedDict([('recall@10', 0.1131), ('mrr@10', 0.1736), ('ndcg@10', 0.1073), ('hit@10', 0.3425), ('precision@10', 0.0699)])
31 Jan 02:03    INFO  test result: OrderedDict([('recall@10', 0.1154), ('mrr@10', 0.1883), ('ndcg@10', 0.1305), ('hit@10', 0.2658), ('precision@10', 0.0747)])


{
    "best_valid_score": 0.1131,
    "valid_score_bigger": true,
    "best_valid_result": {
        "recall@10": 0.1131,
        "mrr@10": 0.1736,
        "ndcg@10": 0.1073,
        "hit@10": 0.3425,
        "precision@10": 0.0699
    },
    "test_result": {
        "recall@10": 0.1154,
        "mrr@10": 0.1883,
        "ndcg@10": 0.1305,
        "hit@10": 0.2658,
        "precision@10": 0.0747
    }
}


### 예측 및 평가
테스트 데이터 로드

In [20]:
# configurations initialization
config = Config(model='LightGCN', dataset="test_data", config_file_list=[f'test_data.yaml'])
config['epochs'] = 1
init_seed(config['seed'], config['reproducibility'])
# logger initialization
init_logger(config)

# dataset filtering
test_dataset = create_dataset(config)
logger.info(test_dataset)

# 성능 측정
a_prob = model.predict(test_dataset).tolist()
a_true = [val for val in test_dataset.inter_feat["rating"]]
a_pred = [round(v) for v in a_prob] 

print("Test data prediction")
print(f" - Accuracy = {100*accuracy_score(a_true, a_pred):.2f}%")
print(f" - ROC-AUC  = {100*roc_auc_score(a_true, a_prob):.2f}%")

31 Jan 02:04    INFO  test_data
The number of users: 90
Average actions of users: 1.0
The number of items: 85
Average actions of items: 1.0595238095238095
The number of inters: 89
The sparsity of the dataset: 98.83660130718954%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp']


Test data prediction


ValueError: Classification metrics can't handle a mix of binary and unknown targets

In [25]:
# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
# breakpoint()
best_valid_score, best_valid_result = trainer.fit(
    train_data, valid_data, saved=True, show_progress=config['show_progress']
)

30 Jan 10:42    INFO  epoch 0 training [time: 0.03s, train loss: 1.3741]
30 Jan 10:42    INFO  epoch 0 evaluating [time: 0.02s, valid_score: 0.017100]
30 Jan 10:42    INFO  valid result: 
recall@10 : 0.0496    mrr@10 : 0.0171    ndcg@10 : 0.02    hit@10 : 0.1029    precision@10 : 0.0103
30 Jan 10:42    INFO  Saving current: saved/LightGCN-Jan-30-2023_10-42-15.pth
30 Jan 10:42    INFO  epoch 1 training [time: 0.02s, train loss: 1.3713]
30 Jan 10:42    INFO  epoch 1 evaluating [time: 0.02s, valid_score: 0.026000]
30 Jan 10:42    INFO  valid result: 
recall@10 : 0.0511    mrr@10 : 0.026    ndcg@10 : 0.0246    hit@10 : 0.1176    precision@10 : 0.0132
30 Jan 10:42    INFO  Saving current: saved/LightGCN-Jan-30-2023_10-42-15.pth
30 Jan 10:42    INFO  epoch 2 training [time: 0.02s, train loss: 1.3681]
30 Jan 10:42    INFO  epoch 2 evaluating [time: 0.02s, valid_score: 0.035800]
30 Jan 10:42    INFO  valid result: 
recall@10 : 0.0547    mrr@10 : 0.0358    ndcg@10 : 0.0319    hit@10 : 0.1471   

Type:           RootLogger
String form:    <RootLogger root (INFO)>
File:           /opt/conda/envs/fastapi/lib/python3.8/logging/__init__.py
Docstring:     
A root logger is not that different to any other logger, except that
it must have a logging level and there is only one instance of it in
the hierarchy.
Init docstring: Initialize the logger with the name "root".
